# 🎯 Object Detection with YOLO & Faster R-CNN

Real-time object detection using state-of-the-art models.

**Level**: Specialized  
**Time**: ~75 minutes

In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Faster R-CNN

In [ ]:
# Load pretrained Faster R-CNN
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights).to(device)
model.eval()

categories = weights.meta['categories']
print(f"Detectable classes: {len(categories)}")
print(f"Examples: {categories[:10]}")

In [ ]:
def detect_objects(model, image_path, threshold=0.5):
    """Run object detection on an image."""
    from torchvision.transforms.functional import to_tensor
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = to_tensor(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        predictions = model(image_tensor)[0]
    
    # Filter by threshold
    mask = predictions['scores'] > threshold
    boxes = predictions['boxes'][mask]
    labels = predictions['labels'][mask]
    scores = predictions['scores'][mask]
    
    return boxes, labels, scores, image

def visualize_detections(image, boxes, labels, scores, categories):
    """Visualize detection results."""
    from torchvision.transforms.functional import to_tensor
    
    image_tensor = (to_tensor(image) * 255).byte()
    label_names = [f"{categories[l]}: {s:.2f}" for l, s in zip(labels.cpu(), scores.cpu())]
    
    result = draw_bounding_boxes(image_tensor, boxes.cpu(), label_names, width=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(result.permute(1, 2, 0))
    plt.axis('off')
    plt.show()

## 2. YOLOv8 (Ultralytics)

In [ ]:
# Install: pip install ultralytics
try:
    from ultralytics import YOLO
    
    yolo_model = YOLO('yolov8n.pt')  # Nano version
    print("✅ YOLOv8 loaded")
    
    # Inference
    # results = yolo_model('image.jpg')
    # results[0].show()
except ImportError:
    print("Install ultralytics: pip install ultralytics")

## 3. Custom Object Detector

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def create_custom_detector(num_classes):
    """Create Faster R-CNN for custom classes."""
    model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# Example: 3 classes (background + 2 objects)
custom_detector = create_custom_detector(num_classes=3)
print("✅ Custom detector ready for training")

## 4. Performance Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['YOLOv8n', 'YOLOv8s', 'YOLOv8m', 'Faster R-CNN', 'RetinaNet'],
    'mAP@50': [37.3, 44.9, 50.2, 42.0, 40.0],
    'Speed (ms)': [6.2, 8.5, 15.0, 45.0, 35.0],
    'Params (M)': [3.2, 11.2, 25.9, 41.8, 36.0],
    'Best For': ['Edge/Mobile', 'Balanced', 'Accuracy', 'Research', 'Dense']
})

print("📊 Object Detection Model Comparison:")
display(comparison)

## 🎯 Key Takeaways
1. YOLO for real-time speed
2. Faster R-CNN for accuracy
3. Fine-tuning pretrained models saves time
4. Export to ONNX/TensorRT for deployment